# Run MethaneMIP scenarios

Use fair calibration v1.4.0, which is appropriate for SSP scenarios with corrected NOx emissions.

In [ ]:
import logging

import matplotlib.pyplot as pl
import numpy as np
import pandas as pd
import xarray as xr

from fair import FAIR
from fair.interface import fill, initialise
from fair.io import read_properties

In [ ]:
logger = logging.getLogger('fair')
logger.setLevel(level=logging.CRITICAL)

In [ ]:
f = FAIR(ch4_method="Thornhill2021")

MethaneMIP scenarios are defined to 2050. CMIP7 might run to 2125, so I have extended to 2130 since emissions are given at 10 years

In [ ]:
f.define_time(1750, 2131, 1)

In [ ]:
scenarios = [
    'ssp119',
    'ssp126',
    'ssp245',
    'ssp370',
    'ssp434',
    'ssp460',
    'ssp534-over',
    'ssp585'
]

In [ ]:
f.define_scenarios(scenarios)

In [ ]:
fair_params_1_4_0_file = '../data/parameters/calibrated_constrained_parameters_1.4.0.csv'

In [ ]:
df_configs = pd.read_csv(fair_params_1_4_0_file, index_col=0)
configs = df_configs.index  # this is used as a label for the "config" axis
f.define_configs(configs)

In [ ]:
fair_species_configs_1_4_0_file = '../data/parameters/species_configs_properties_1.4.0.csv'

In [ ]:
species, properties = read_properties(filename=fair_species_configs_1_4_0_file)
f.define_species(species, properties)

In [ ]:
f.allocate()

In [ ]:
f.fill_from_csv(
#    emissions_file='../data/emissions/methane-emissions-scenarios.csv',
    forcing_file='../data/forcing/volcanic_solar.csv',
)

In [ ]:
# I was lazy and didn't convert emissions to CSV, so use the old clunky method of importing from netCDF

da_emissions = xr.load_dataarray("../data/emissions/ssp_emissions_1750-2500.nc")
output_ensemble_size = 841
da = da_emissions.loc[dict(config="unspecified")][:381, ...]
fe = da.expand_dims(dim=["config"], axis=(2))
f.emissions = fe.drop("config") * np.ones((1, 1, output_ensemble_size, 1))

In [ ]:
fill(
    f.forcing,
    f.forcing.sel(specie="Volcanic") * df_configs["forcing_scale[Volcanic]"].values.squeeze(),
    specie="Volcanic",
)
fill(
    f.forcing,
    f.forcing.sel(specie="Solar") * df_configs["forcing_scale[Solar]"].values.squeeze(),
    specie="Solar",
)

In [ ]:
f.fill_species_configs(fair_species_configs_1_4_0_file)
f.override_defaults(fair_params_1_4_0_file)

In [ ]:
initialise(f.concentration, f.species_configs["baseline_concentration"])
initialise(f.forcing, 0)
initialise(f.temperature, 0)
initialise(f.cumulative_emissions, 0)
initialise(f.airborne_emissions, 0)
initialise(f.ocean_heat_content_change, 0)

In [ ]:
f.run()

In [ ]:
colors = {
    "ssp119": "#00a9cf",
    "ssp126": "#003466",
    "ssp245": "#f69320",
    "ssp370": "#df0000",
    "ssp434": "#2274ae",
    "ssp460": "#b0724e",
    "ssp534-over": "#92397a",
    "ssp585": "#980002",
}

In [ ]:
for scenario in scenarios:
    pl.plot(np.arange(2000, 2131), f.concentration.sel(specie='CH4', scenario=scenario, timebounds=np.arange(2000, 2131)), color=colors[scenario]);
# pl.xlim(250, 300)

In [ ]:
pd.DataFrame(
    f.concentration.sel(specie='CH4').interp(timebounds=f.timepoints).median(dim='config'),
    index=f.timepoints,
    columns=f.scenarios
).to_csv('../output/ssp-ch4-concentration-medians.csv')

In [ ]:
for scenario in scenarios:
    pl.plot(np.arange(2000, 2131), 10 * f.alpha_lifetime.sel(specie='CH4', scenario=scenario, timebounds=np.arange(2000, 2131)), color=colors[scenario]);

In [ ]:
weights = np.zeros((382, 8, 841))
weights[100, :, :] = 0.5
weights[101:151, :, :] = 1
weights[151, :, :] = 0.5
weights = xr.DataArray(
    weights, 
    dims=f.temperature.sel(layer=0).dims, 
    coords=f.temperature.sel(layer=0).coords
)

In [ ]:
temp_rel_1850_1900 = (
    f.temperature.sel(layer=0) - 
    f.temperature.sel(layer=0, timebounds=np.arange(1850, 1902)).weighted(weights).mean(dim="timebounds")
).interp(timebounds=f.timepoints)

In [ ]:
temp_rel_1850_1900

In [ ]:
ds = xr.Dataset(
    data_vars = dict(
        temperature = (["timepoints", "scenario", "config"], temp_rel_1850_1900.data),
    ),
    coords = dict(
        timepoints = f.timepoints,
        config = f.configs,
        scenario = f.scenarios,
    ),
)

In [ ]:
ds.to_netcdf('../output/ssp-temperature-relative-to-1850-1900.nc')